<a href="https://colab.research.google.com/github/Toadoum/NLP-AMMI/blob/main/Sakayo_Toadoum_Sari_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [22]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [23]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/drive/MyDrive/Colab Notebooks/NLP_Week_1_Labs_2022/session3/train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/drive/MyDrive/Colab Notebooks/NLP_Week_1_Labs_2022/session3/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [25]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount

    ## FILL CODE
    for idx1, words in enumerate(data):
        for idx2, word in enumerate(words):
            if word not in vocab.keys():
                data[idx1][idx2] = '<unk>'
            elif vocab[word] < mincount:
                data[idx1][idx2] = '<unk>'
    
    return data

In [26]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


In [10]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        total_number_words += len(sentence)
        for gram in range(1, n+1):
            for i in range(len(sentence)-gram+1):
#                 ipdb.set_trace()
                counts[sentence[i:(gram+i-1)]][sentence[gram+i-1]]+=1
                           

    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL CODE
        for target in counts[context].keys():
            prob[context][target] = counts[context][target]/(sum(counts[context].values()))

    return prob

In [27]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [35]:
import sys

In [42]:
def get_prob(model, context, w):
    sys.setrecursionlimit(2000)
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    if model[context][w] != 0:
        return model[context][w]
    else:
        return 0.4*get_prob(model, context[1:], w)

In [43]:
def perplexity(model, data, n):
    sys.setrecursionlimit(2000)
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE

    perp = []
        
    for sentence in data:
        sentence = tuple(sentence)
        perp_s = 1.0
        for i in range(len(sentence)):
            if (n+i-1) < len(sentence):                
                perp_s *= 1.0/get_prob(model, sentence[i:(n+i-1)], sentence[n+i-1])
            
        perp.append(perp_s**(1/len(sentence)))
                
    return np.mean(perp)

In [44]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

RecursionError: ignored

In [45]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    if sum(model[context].values()) != 0:
        return context
    else :
#         ipdb.set_trace()
        return get_proba_distrib(model, context[1:])

In [46]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    
    i = 0
    while True :
        
      avail_context = get_proba_distrib(model,tuple(sentence[max(i+1-n, 0):i+1]))
#         ipdb.set_trace()
      a = list(model[avail_context].keys())
      prob = list(model[avail_context].values())
      current_pred = np.random.choice(a, 1, \
                                    p = prob)
      sentence.append(current_pred[0])
      i+=1
      if current_pred[0] == '</s>':
          break
    
    return sentence

In [47]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'do', 'everything', 'they', 'said', "they're", 'having', 'the', 'room', 'in', 'the', 'manager', 'before', 'then.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.